In [1]:
pip install xgboost

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install bayesian-optimization

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lightgbm

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scikit-optimize==0.8.1

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scikit-learn==0.23.2

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install catboost

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pickle
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import matplotlib.pyplot as plt

import seaborn as sns

import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import  LogisticRegression
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score,GridSearchCV
from skopt import BayesSearchCV

from sklearn.metrics import roc_curve, auc

from sklearn.model_selection import train_test_split

%matplotlib inline
plt.style.use('fivethirtyeight')

## Part 2: Supervised Learning Model


## Load and clean the train and test dataset

In [8]:

# I have renamed Udacity_MAILOUT_052018_TRAIN.csv and Udacity_MAILOUT_052018_TEST.csv for more readability
# load data from csv files to make operation on them
# missing values found in section of missing values and set it here for furure use
missing_values = ["X", "XX"]

#read csv file of mailout train with changing missing values to nan
mailout_train = pd.read_csv('../data/mail_train.csv',sep=';',na_values=missing_values)
#read csv file of mailout test with changing missing values to nan
mailout_test = pd.read_csv('../data/mail_test.csv',sep=';',na_values=missing_values)


In [9]:
#check if mailout train has more than 80% missing values

#sum of mailout_train null value
mailout_train_null_sum=mailout_train.isnull().sum()

#check percent of mailout train missing value
missing_columns=mailout_train_null_sum/len(mailout_train)

#list which features have missing value more than 80 percent
missing_percent=0.80
mailout_train[missing_columns[missing_columns > missing_percent].index]


,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
42957,NaN,NaN,NaN,NaN
42958,NaN,NaN,NaN,NaN
42959,NaN,NaN,NaN,NaN
42960,NaN,NaN,NaN,NaN


In [10]:
#check if mailout test has more than 80% missing values

#sum of mailout_test null value
mailout_test_null_sum=mailout_test.isnull().sum()

#check percent of mailout test missing value
missing_columns=mailout_test_null_sum/len(mailout_test)

#list which features have missing value more than 80 percent
missing_percent=0.80
mailout_test[missing_columns[missing_columns > missing_percent].index]


,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
42828,NaN,NaN,NaN,NaN
42829,NaN,NaN,NaN,NaN
42830,14.0,17.0,NaN,NaN
42831,NaN,NaN,NaN,NaN


In [11]:
#I have renamed DIAS Attributes - Values 2017.xlsx to attributes
# use openpyxl as an engine for reading xlsx files
# as far as first row of attributes file is empty I set skip first row of attributes
attributes = pd.read_excel('../data/attributes.xlsx', engine='openpyxl', skiprows = 1)


In [12]:
#first I have to list all values that recognize as unknow
unkown_list= attributes['Meaning'].where(attributes['Meaning'].str.contains('unknown')).value_counts().index

In [13]:
# make a list that have unknown values of each attribute
unkown_attributes_list = attributes[attributes['Meaning'].isin(unkown_list)]

In [14]:
def unknown_to_nan(dataframe):
   
    # rotate through all of unkown attributes
    for row in unkown_attributes_list.iterrows():
        
        # list the missed Value attributes from attributs excel file
        missed_values_list = row[1]['Value']
        
        # list the attributes from attributs excel file
        attribute = row[1]['Attribute']

        # to pass if columns is in dataframe
        if attribute not in dataframe.columns:
            
            continue

        # to pass if value is integer or string
        if isinstance(missed_values_list, int):
            
            dataframe[attribute].replace(missed_values_list, -1, inplace=True)
        # to pass if value is String   
        elif isinstance(missed_values_list, str):
            
            #parses the expression to make it as integer -1 with unkown
            eval("dataframe[attribute].replace([" + missed_values_list + "], -1, inplace=True)")
            
            dataframe[attribute].replace(missed_values_list, -1, inplace=True)

    return dataframe

In [15]:
#change all mailout_train unkown value to fill with -1
mailout_train = unknown_to_nan(mailout_train)


In [16]:
#change all mailout_train unkown value to fill with -1
mailout_test = unknown_to_nan(mailout_test)


In [17]:
#cast datatype to float
def values_type_to_float(dataframe,column):
    
    dataframe[column] = dataframe[column].astype('float')

    
    return dataframe

In [18]:
#map the value to numerical value  
def values_map(dataframe,feature_old,feature_new,dictionary):
    
    dataframe[feature_new] = dataframe.loc[:, feature_old].map(dictionary)
    
    return dataframe

In [19]:
# make feature to have just year to mange it as numerical value
def year_extractor(dataframe,column):
    
    dataframe['EINGEFUEGT_AM'] = pd.to_datetime(dataframe['EINGEFUEGT_AM']).dt.year
    
    return dataframe


In [20]:
def drop_features(dataframe,features):

    dataframe.drop(features, axis=1,inplace=True)
    
    return dataframe

In [21]:
#step by step cleaning and transforming process
def clean_transform(dataframe,dataframe_name):
    
    #dataframe= values_replace(dataframe)
    print(dataframe_name,'replace object values : done')
    
    dataframe= values_type_to_float(dataframe,'CAMEO_DEUG_2015')
    print(dataframe_name,'cast values to float : done')
    
    dataframe= values_type_to_float(dataframe,'CAMEO_INTL_2015')
    print(dataframe_name,'cast values to float : done')
    

    dataframe= values_map(dataframe,'OST_WEST_KZ','OST_WEST_KZ_E',{'W': 0, 'O': 1})
    print(dataframe_name,'OST_WEST_KZ mapping : done')

    
    dataframe= year_extractor(dataframe,'EINGEFUEGT_AM')
    print(dataframe_name,'EINGEFUEGT_AM year extraction : done')

    
    dataframe= drop_features(dataframe,['LNR', 'D19_LETZTER_KAUF_BRANCHE', 'OST_WEST_KZ', 'CAMEO_DEU_2015'])
    print(dataframe_name,'drop some features : done')

    
    dataframe= values_map(dataframe,'PRAEGENDE_JUGENDJAHRE','PRAEGENDE_JUGENDJAHRE_DECADE',{1: 1, 2: 1, 3: 2, 4: 2, 5: 3, 6: 3, 7: 3, 8: 4, 9: 4, 10: 5,11: 5, 12: 5, 13: 5, 14: 6,15: 6, 0: np.nan})
    print(dataframe_name,'PRAEGENDE_JUGENDJAHRE mapping : done')

    
    dataframe= values_map(dataframe,'PRAEGENDE_JUGENDJAHRE','PRAEGENDE_JUGENDJAHRE_MOVEMENT',{1: 0, 2: 1, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 0, 9: 1, 10: 0, 11: 1, 12: 0, 13: 1, 14: 0, 15: 1, 0: np.nan})
    print(dataframe_name,'PRAEGENDE_JUGENDJAHRE mapping : done')

    
    dataframe= drop_features(dataframe,['PRAEGENDE_JUGENDJAHRE'])
    print(dataframe_name,'drop PRAEGENDE_JUGENDJAHRE as an old feature : done')


    print(dataframe_name,'cleaning and transforming finished')
    return dataframe

In [22]:
#start the proccess of clean mailout_train dataframe from object datatype and transform some features to new feature
mailout_train = clean_transform(mailout_train,'mailout_train')

mailout_train replace object values : done
mailout_train cast values to float : done
mailout_train cast values to float : done
mailout_train OST_WEST_KZ mapping : done
mailout_train EINGEFUEGT_AM year extraction : done
mailout_train drop some features : done
mailout_train PRAEGENDE_JUGENDJAHRE mapping : done
mailout_train PRAEGENDE_JUGENDJAHRE mapping : done
mailout_train drop PRAEGENDE_JUGENDJAHRE as an old feature : done
mailout_train cleaning and transforming finished


In [23]:
#start the proccess of clean mailout_test dataframe from object datatype and transform some features to new feature
mailout_test = clean_transform(mailout_test,'mailout_test')

mailout_test replace object values : done
mailout_test cast values to float : done
mailout_test cast values to float : done
mailout_test OST_WEST_KZ mapping : done
mailout_test EINGEFUEGT_AM year extraction : done
mailout_test drop some features : done
mailout_test PRAEGENDE_JUGENDJAHRE mapping : done
mailout_test PRAEGENDE_JUGENDJAHRE mapping : done
mailout_test drop PRAEGENDE_JUGENDJAHRE as an old feature : done
mailout_test cleaning and transforming finished


In [24]:
#here we create our train and test dataframe and dataframe of actual output for test it with our model

X_train = mailout_train

X_test = mailout_test

y_train = X_train['RESPONSE']


X_train.drop(['RESPONSE'], axis = 1, inplace = True)


# catboost optimzer

In [35]:
import pprint

def performance_report(BayesSearchCV, X_train_set, y_taget_set, experiment, callbacks):
  
    #define start time to check the time for each check
    start_time = time()
    
    if callbacks:
        BayesSearchCV.fit(X_train_set, y_taget_set, callback=callbacks)
    else:
        BayesSearchCV.fit(X_train_set, y_taget_set)
        
    bs_d_cv=pd.DataFrame(BayesSearchCV.cv_results_)
    
    best_cv_score = BayesSearchCV.best_score_
    
    best_cv_score_std = bs_d_cv.iloc[BayesSearchCV.best_index_].std_test_score
    
    best_BayesSearchCV_params = BayesSearchCV.best_params_
    
    print(experiment)
    print("elapsed %.2f seconds" %(time() - start_time))
    print("measured candidates: %d" % len(BayesSearchCV.cv_results_['params']))
    print("best CV score: %.3f" % best_cv_score)
    print(best_cv_score_std)
       
    print('Best tunned params:')
    
    pprint.pprint(best_BayesSearchCV_params)
    
    print()
    
    return best_params

In [36]:
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score

roc_auc = make_scorer(roc_auc_score, greater_is_better=True, needs_threshold=True)
Stratified_K_Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [37]:
classifier = CatBoostClassifier(thread_count=2,
                         loss_function='Logloss',
                         od_type = 'Iter',
                         verbose= False
                        )

In [38]:

from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer
from time import time

#define limit of Bayes Search CV hyper parameter
bayes_tunning_bounds = {'iterations': Integer(10, 1000),
                 'depth': Integer(1, 8),
                 'learning_rate': Real(0.01, 1.0, 'log-uniform'),
                 'random_strength': Real(1e-9, 10, 'log-uniform'),
                 'bagging_temperature': Real(0.0, 1.0),
                 'border_count': Integer(1, 255),
                 'l2_leaf_reg': Integer(2, 30),
                 'scale_pos_weight':Real(0.01, 1.0, 'uniform')}

In [39]:
# define BayesSearchCV for optimizing
optimizer = BayesSearchCV(clf,
                    search_spaces,
                    scoring=roc_auc,
                    cv=skf,
                    n_iter=100,
                    n_jobs=1,  
                    return_train_score=False,
                    refit=True,
                    optimizer_kwargs={'base_estimator': 'GP'},
                    random_state=42)

In [40]:
best_params_report = performance_report(optimizer=opt, X=X_train, y=y_train,title='CatBoost', 
                                        #define callback
                          callbacks=[VerboseCallback(100),DeadlineStopper(60*10)])

Iteration No: 1 started. Searching for the next optimal point.
Iteration No: 1 ended. Search finished for the next optimal point.
Time taken: 52.1063
Function value obtained: -0.7668
Current minimum: -0.7668
Iteration No: 2 started. Searching for the next optimal point.
Iteration No: 2 ended. Search finished for the next optimal point.
Time taken: 87.7804
Function value obtained: -0.7689
Current minimum: -0.7689
Iteration No: 3 started. Searching for the next optimal point.
Iteration No: 3 ended. Search finished for the next optimal point.
Time taken: 37.1979
Function value obtained: -0.7700
Current minimum: -0.7700
Iteration No: 4 started. Searching for the next optimal point.
Iteration No: 4 ended. Search finished for the next optimal point.
Time taken: 92.1591
Function value obtained: -0.7726
Current minimum: -0.7726
Iteration No: 5 started. Searching for the next optimal point.
Iteration No: 5 ended. Search finished for the next optimal point.
Time taken: 88.0796
Function value obt

# catboost

In [41]:
number_of_features = 366

#Calculate Feature Importance
model_features_importance = CatBoostClassifier( verbose = False).fit( X_train, y_train)
features_importance = pd.DataFrame({'features':X_train.columns, 'importance' : model_features_importance.feature_importances_/ model_features_importance.feature_importances_.sum()}).sort_values(['importance'], ascending = False)


In [54]:
#reduce the ftrain and test data based on the important features
X_train_kahesh = X_train[features_importance['features'].head(number_of_features).to_list()]
X_test_kahesh = X_test[features_importance['features'].head(number_of_features).to_list()]

In [42]:
mailout_test_LNR= pd.read_csv('../data/mail_test.csv',sep=';',na_values=missing_values)
mailout_test_LNR=unknown_to_nan(mailout_test_LNR)

In [44]:
params={'bagging_temperature': 0.8123959883573634,
 'border_count': 45,
 'depth': 5,
 'iterations': 805,
 'l2_leaf_reg': 17,
 'learning_rate': 0.01552065618292981,
 'random_strength': 0.0361414296804267,
 'scale_pos_weight': 0.8739040497768427}

In [45]:
#use the dictionaty with optimized parameters result of the BayesSearchCV
kaggle_model = CatBoostClassifier(**params)

#fit kaggle model with reduced x and y
kaggle_model.fit(X_train_kahesh, y_train)

y_sub = kaggle_model.predict_proba(X_test_kahesh)[:, 1]


0:	learn: 0.6595221	total: 14.9ms	remaining: 12s
1:	learn: 0.6276022	total: 28.7ms	remaining: 11.5s
2:	learn: 0.5953978	total: 45.1ms	remaining: 12.1s
3:	learn: 0.5668900	total: 61.6ms	remaining: 12.3s
4:	learn: 0.5398439	total: 73ms	remaining: 11.7s
5:	learn: 0.5140904	total: 89.6ms	remaining: 11.9s
6:	learn: 0.4898201	total: 105ms	remaining: 12s
7:	learn: 0.4669367	total: 121ms	remaining: 12.1s
8:	learn: 0.4451459	total: 136ms	remaining: 12s
9:	learn: 0.4246341	total: 150ms	remaining: 11.9s
10:	learn: 0.4052704	total: 163ms	remaining: 11.8s
11:	learn: 0.3869866	total: 177ms	remaining: 11.7s
12:	learn: 0.3698339	total: 194ms	remaining: 11.8s
13:	learn: 0.3535165	total: 211ms	remaining: 11.9s
14:	learn: 0.3381384	total: 228ms	remaining: 12s
15:	learn: 0.3235660	total: 243ms	remaining: 12s
16:	learn: 0.3098250	total: 260ms	remaining: 12s
17:	learn: 0.2968017	total: 275ms	remaining: 12s
18:	learn: 0.2825746	total: 293ms	remaining: 12.1s
19:	learn: 0.2710700	total: 310ms	remaining: 12.2s


In [56]:
#create submit file for kaggle
submission = pd.DataFrame({'LNR': mailout_test_LNR['LNR'], 'RESPONSE': y_sub})
#save dataframe to csv file for submit
submission.to_csv('submission_catboost.csv', index=False)